<a href="https://colab.research.google.com/github/siherm/PUMA/blob/main/PUMA_Duplicate_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
%pip install pydantic

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\jomar\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [59]:
import dictdiffer
import os
from pprint import  pprint
from pyPUMA import PUMACollection, PUMAEntry

## Configuration

In [42]:
# 1 - Fetch data from PUMA (Replace if necessary or add to environment vars)
username = "jo-hollenbeck"
token = "5efe8593931e3c325f9399812b1de5da"

## Utilizing the PUMACollection
### Application: Find duplicates

In [66]:
# Fetch remote collection from Simtech group
collection = PUMACollection.from_web_api(username, token)

# Find all duplicates and sort into data structure
duplicates = collection.gather_duplicates("doi")

for dup in duplicates:
    print(dup)

10.1007/s11242-020-01506-w
10.1021/acs.jpcc.0c08870
10.1021/acs.jctc.0c00348
10.1002/rnc.3360
10.1007/s11242-020-01445-6
10.1021/acs.jpca.9b08239
10.1109/med.2015.7158818
10.1080/10255842.2013.853754
10.2218/iclass.2021.5815
10.1109/tvcg.2019.2961893
10.1007/978-3-030-80602-6_17
10.1007/s00332-021-09755-9
10.1016/j.ifacol.2015.09.296
10.1016/j.ijsolstr.2017.04.012
10.1016/j.sysconle.2013.04.001
10.1016/j.compfluid.2020.104825
10.1016/j.ifacol.2015.09.437
10.1016/j.jcp.2021.110475
10.1109/cdc40024.2019.9029169
10.1186/s42492-021-00090-0
10.1109/escience.2015.69
10.1144/petgeo2016-068
10.1002/fld.4065
10.1016/j.automatica.2014.10.002
10.1002/2016wr018969
10.1021/acs.jctc.1c00527
10.1002/nme.6869
10.1002/rnc.3751
10.1111/gwmr.12497
10.1109/ecc.2015.7330585
10.1016/j.ifacol.2018.11.138
10.1109/ecc.2015.7331027
10.1145/3399715.3399814
10.1109/lcsys.2017.2718556
10.1002/rnc.3042
10.3182/20130902-5-de-2040.00082
10.1007/s10494-016-9704-y
10.1007/s10822-021-00439-w
10.1080/00207179.2020.174528

In [67]:
# Get an example
record = duplicates["10.17192/bfdm.2022.1.8412"]


In [68]:
for tup in dictdiffer.diff(record[0].dict(exclude={"bibtex"}), record[1].dict(exclude={"bibtex"})):
    print(tup)

('change', 'journal', ('Bausteine Forschungsdatenmanagement', None))


In [69]:
authors = record[0].author.split(" and ")

In [70]:
for author in authors:
    surname, name = author.split(",")
    
    print(name, surname)

 Christian Brecher
 Michael R. Buchmeiser
 Andreas Burkert
 Marius R. Busemeyer
 Stephan Conermann
 Thomas Ertl
 Michael Friedrich
 Rainer Helmig
 Volker Hohmann
 Andrew James Johnston
 Birger Kollmeier
 Matthew Larkum
 Jan Louis
 Achim Menges
 Uwe Morgner
 Jürgen Müller
 Carien Niessen
 Mario Ohlberger
 Wolfgang Schäffner
 Piet Schmidt
 Dietmar Schmitz
 Werner Seeger
 Detlef Stammer
 Arne Thomas
 Anita Traninger
 Martin Wegener
 Julien Colomb
 Sibylle Hermann
 Jenny Kopsch-Xhema
 Jan Range
 Bernd Flemisch


In [48]:
# Use the PUMAEntry method to merge 
print(PUMAEntry.from_multiple_entries(entries=[record[0], record[1]]).json(indent=2))

['Beck, Andrea D. ', ' Flad, David G. ', ' Tonh{\\"a}user, Claudia ', ' Gassner, Gregor ', ' Munz, Claus-Dieter']
['Beck, A. ', ' Flad, D. ', ' Tonhäuser, C. ', ' Gassner, G. ', ' Munz, C.-D.']
{
  "title": "On the Influence of Polynomial De-aliasing on Subgrid Scale Models",
  "user": "simtech",
  "author": "Beck, Andrea D. and Flad, David G. and Tonh{\\\"a}user, Claudia and Gassner, Gregor and Munz, Claus-Dieter",
  "entrytype": "article",
  "group": "SimTech",
  "tag": [
    "EXC310",
    "from:katharinafuchs",
    "imported",
    "pn1",
    "from:simtechpuma"
  ],
  "journal": "Flow, Turbulence and Combustion",
  "preprint_id": null,
  "url": "https://doi.org/10.1007/s10494-016-9704-y",
  "doi": "10.1007/s10494-016-9704-y",
  "isbn": null,
  "issn": "1573-1987",
  "misc": null
}


In [49]:
print(PUMAEntry.from_multiple_entries.__doc__)

Merges multiple entries to a single one

        This method is used to merge multiple entries to a single one in the
        duplicate analysis given within PyPUMA. It should be noted, that the
        static fields from the first entry will be preferred if not specified
        in the recursion dictionary. Fields that are lists will be added and
        duplicate values merged to a single.

        In order to apply preferences, a recursion dictionary can be given
        from which the fields value that matches will be used. For instance,
        if the entry_type attribute is mixed but specified as 'article' the
        latter will be used for the new entry. If no values apply to the
        recursion, the first will be used.


        Args:
          entries (list[PUMAEntry]): List of PUMAEntries that will be merged
          recursion (dict): Prefrences that will be used to merge.
        


In [50]:
import re

In [51]:
def filter_regex(result: list[tuple]):
    return [
        tup for tup in result
        if list(filter(None, tup))
    ]

In [52]:
regex = r"[pn|PN](\d*)(-\w?)?"
regex = re.compile(regex)

In [53]:
string = "pn<7>"
string = re.sub(r">|<", "", string)

In [54]:
filter_regex(regex.findall(string))

[('7', '')]

In [55]:
tags = []
for record in collection:
    for tag in record.tag:
        if "graduateschool" in tag.lower():
            tags.append(tag.lower())
    

In [56]:
set(tags)

{'graduateschool'}

In [57]:
collection.records[0].__dict__.keys()

dict_keys(['bibtex', 'title', 'user', 'author', 'entrytype', 'group', 'tag', 'journal', 'preprint_id', 'url', 'doi', 'isbn', 'issn', 'misc'])

In [58]:
bool(("", ""))

True